In [1]:
#m Se importan las librerias necesarias
from datetime import datetime
import pandas as pd
import numpy as np
import datetime
import collections
import pandas_datareader.data as web

In [2]:
# Se importa la librería para importar los historiales de metatraderfrom datetime import datetime
import MetaTrader5 as mt5

In [3]:
local_exe_berlin = 'C:\Program Files\FxPro - MetaTrader 5\terminal64.exe'
local_exe_jorge ='C:\Program Files\MetaTrader 5\terminal64.exe'
# Fechas de descarga
fi = datetime.datetime(2022, 2, 14, 0, 0) #Fecha de inicio de historicos
ff = datetime.datetime(2022, 3, 22, 0, 0)# Fecha final 


La siguiente función será utilizada para poder extraer y leer los historicos de las operaciones de las distintas cuentas en Metratrade5-

In [4]:
def f_leer_archivo(numcue: int, contra: str, direc: str, fi, ff):
    usuario = mt5.f_init_login(numcue, contra, direc)
    usuinfo = mt5.f_acc_info(mt5_client)
    hist = mt5.f_hist_trades(param_ct=usuario, usuinfo=fi, hist=ff)#Precios historicos
    # Utilizando excel
    return hist

In [5]:
def f_leer_archivo(nombre):
    # Numero de posiciones que tuviste debe de ser cambiado
    posiciones = pd.read_csv(nombre,skiprows = 6,nrows = 15)
    for i in range(len(posiciones)):
        posiciones.loc[i,"Price"] = (posiciones.loc[i,"Price"]).replace(" ","")
        posiciones.loc[i,"Price.1"] = (posiciones.loc[i,"Price.1"]).replace(" ","")
        posiciones.loc[i,"Profit"] = (posiciones.loc[i,"Profit"]).replace(" ","")
    posiciones = posiciones.astype({'Price':'float'})
    posiciones = posiciones.astype({'Price.1':'float'})
    posiciones = posiciones.astype({'Profit':'float'})
    #Cambiar nombre de las commodities
    posiciones = posiciones.replace("WTI","WTICOUSD")
    posiciones = posiciones.replace("PALLADIUM","XPDUSD")
    posiciones = posiciones.replace("PLATINUM","XPTUSD")
    return posiciones

In [36]:
def f_columnas_tiempos(param_data):
    param_data['Time'] =  pd.to_datetime(param_data['Time'])
    param_data['Time.1'] =  pd.to_datetime(param_data['Time.1'])
    param_data.rename(columns = {'Time':'Opentime', 'Time.1':'Closetime'}, inplace = True)
    param_data["Tiempo"] = (param_data["Closetime"] - param_data["Opentime"]).dt.total_seconds()
    return param_data

In [8]:
def f_leer_pip():
    doc_pips= pd.read_csv("instruments_pips.csv")
    for i in range(len(doc_pips)):
        doc_pips.loc[i,"Instrument"] = (doc_pips.loc[i,"Instrument"]).replace("_","")
    doc_pips = doc_pips.set_index("Instrument")
    new_doc_pips = doc_pips.filter(["TickSize"])
    # Añadir instrumentos que no estan en el excel
    new_doc_pips.loc["BITCOIN"] = .01
    new_doc_pips.loc["ETHEREUM"] = .01
    return new_doc_pips

In [10]:
def f_pip_size(instrumento: str):
    return 1/archivo_pip.loc[instrumento,"TickSize"]

In [12]:
def f_columnas_pips(posiciones):
    # Añadir el pipsize de cada instrumento
    pip_size = []
    pip = []
    for i in range(len(posiciones)):
        pip_size.append(f_pip_size(posiciones["Symbol"][i]))
    posiciones["Pip_size"] = pip_size
    # Cantidad de pips resultantes de cada operación 
    for i in range(len(posiciones)):
        if posiciones["Type"][i] == "sell":
            pip.append((posiciones["Price"][i] - posiciones["Price.1"][i])*posiciones["Pip_size"][i])
        else:
            pip.append((posiciones["Price.1"][i] - posiciones["Price"][i])*posiciones["Pip_size"][i])
    posiciones["Pip"] = pip
    posiciones["Pips_acm"] = posiciones["Pip"].cumsum()
    posiciones["Profit_acm"] = posiciones["Profit"].cumsum()
    return posiciones

In [15]:
def f_estadisticas_ba(posiciones):
    lista_medida = ["Ops totales","Ganadoras","Ganadoras_c","Ganadoras_v","Perdedoras","Perdedoras_c","Perdedoras_v","Mediana(Profit)",
                   "Mediana(Pips)","r_efectividad","r_proporcion","r_efectividad_c","r_efectividad_v"]
    lista_descripcion = ["Operaciones totales","Operaciones ganadoras","Operaciones ganadoras de compra","Operaciones ganadoras de venta","Operaciones perdedoras",
                         "Operaciones perdedoras de compra","Operaciones perdedoras de venta","Mediana de profit de operaciones","Mediana de pips de operaciones",
                         "Ganadoras Totales/Operaciones Totales","Ganadoras Totales/Perdedoras Totales","Ganadoras compras/Operaciones Totales",
                         "Ganadoras ventas/Operaciones Totales"]
    data = {"Valor":[len(posiciones),
                     len(posiciones.loc[posiciones['Profit'] > 0]),
                     len(posiciones.loc[(posiciones['Profit'] > 0) & (posiciones["Type"] == "buy")]),
                    len(posiciones.loc[(posiciones['Profit'] > 0) & (posiciones["Type"] == "sell")]),
                     len(posiciones.loc[posiciones['Profit'] < 0]),
                     len(posiciones.loc[(posiciones['Profit'] < 0) & (posiciones["Type"] == "buy")]),
                     len(posiciones.loc[(posiciones['Profit'] < 0) & (posiciones["Type"] == "sell")]),
                     posiciones['Profit'].median(),
                     posiciones['Pip'].median(),
                     len(posiciones.loc[posiciones['Profit'] > 0])/len(posiciones),
                     len(posiciones.loc[posiciones['Profit'] > 0])/len(posiciones.loc[posiciones['Profit'] < 0]),
                     len(posiciones.loc[(posiciones['Profit'] > 0) & (posiciones["Type"] == "buy")])/len(posiciones),
                     len(posiciones.loc[(posiciones['Profit'] > 0) & (posiciones["Type"] == "sell")])/len(posiciones)
                    ],
           "Descripcion": lista_descripcion,
           "Medida": lista_medida}
    estadisticas_ba = pd.DataFrame(data)
    estadisticas_ba = estadisticas_ba.set_index("Medida")
    
    #Segunda parte de la función 
    
    # Obtener todos los instrumentos operados
    symbols = list(posiciones["Symbol"].unique())
    symbol_ratio = []
    for i in symbols:
        symbol_ratio.append(len(posiciones.loc[(posiciones['Profit'] > 0) & (posiciones["Symbol"] == i)])/len(posiciones.loc[posiciones['Symbol'] == i]))
    ranking = {"Symbol":symbols,"Rank": symbol_ratio}
    ranking = pd.DataFrame(ranking)
    ranking = ranking.set_index("Symbol")
    ranking = ranking.sort_values(by = ["Rank"],ascending = False)
    
    dict_estadisticas = {"df_1_tabla": estadisticas_ba,"df_2_ranking":ranking} 
    
    return dict_estadisticas

In [37]:
archivo_pip = f_leer_pip()

In [38]:
posiciones = f_leer_archivo("historico_jorge.csv")

In [39]:
f_columnas_pips(posiciones)

,Time,Position,Symbol,Type,Volume,Price,S / L,T / P,Time.1,Price.1,Commission,Swap,Profit,Unnamed: 13,Unnamed: 14,Pip_size,Pip,Pips_acm,Profit_acm
0,2022.03.01 02:25:12,49329411,ETHEREUM,sell,10,2934.92000,2 984.92,2 834.92,2022.03.01 15:06:45,2986.93000,0.0,0.00,-520.10,NaN,NaN,100.0,-5201.0,-5201.0,-520.10
1,2022.03.01 15:24:12,49350256,EURUSD,buy,10,1.11441,1.11341,1.11741,2022.03.01 17:10:21,1.11340,0.0,0.00,-1010.00,NaN,NaN,10000.0,-10.1,-5211.1,-1530.10
2,2022.03.01 15:45:08,49351425,BITCOIN,sell,20,44477.04000,44 777.04,44 177.04,2022.03.01 16:33:16,44173.06000,0.0,0.00,6079.50,NaN,NaN,100.0,30398.0,25186.9,4549.40
3,2022.03.03 08:07:11,49432596,EURGBP,buy,1,0.82804,0.82604,0.83104,2022.03.04 09:25:12,0.82604,0.0,- 5.35,-266.51,NaN,NaN,10000.0,-20.0,25166.9,4282.89
4,2022.03.03 08:23:08,49432860,WTICOUSD,sell,5,112.39500,113.614,109.956,2022.03.03 09:26:06,113.61800,0.0,0.00,-6115.00,NaN,NaN,100.0,-122.3,25044.6,-1832.11
5,2022.03.03 08:37:33,49433058,XPDUSD,sell,1,2681.38000,2 705.60,2 657.16,2022.03.03 09:04:31,2707.34000,0.0,0.00,-2596.00,NaN,NaN,100.0,-2596.0,22448.6,-4428.11
6,2022.03.03 08:51:02,49433247,GBPUSD,buy,1,1.34097,1.33910,1.34237,2022.03.03 09:47:13,1.33908,0.0,0.00,-189.00,NaN,NaN,10000.0,-18.9,22429.7,-4617.11
7,2022.03.03 08:57:11,49433357,USDMXN,buy,2,20.64708,20.58859,20.67110,2022.03.03 10:24:18,20.67181,0.0,0.00,239.26,NaN,NaN,10000.0,247.3,22677.0,-4377.85
8,2022.03.03 08:59:04,49433389,BITCOIN,buy,10,43500.96000,43 025.18,43 867.18,2022.03.03 15:56:30,43869.41000,0.0,0.00,3684.50,NaN,NaN,100.0,36845.0,59522.0,-693.35
9,2022.03.03 09:00:40,49433433,XPTUSD,sell,1,1082.82000,1 093.68,1 071.96,2022.03.03 10:57:49,1093.64000,0.0,0.00,-1082.00,NaN,NaN,100.0,-1082.0,58440.0,-1775.35


In [40]:
f_columnas_tiempos(posiciones)

,Opentime,Position,Symbol,Type,Volume,Price,S / L,T / P,Closetime,Price.1,Commission,Swap,Profit,Unnamed: 13,Unnamed: 14,Pip_size,Pip,Pips_acm,Profit_acm,Tiempo
0,2022-03-01 02:25:12,49329411,ETHEREUM,sell,10,2934.92000,2 984.92,2 834.92,2022-03-01 15:06:45,2986.93000,0.0,0.00,-520.10,NaN,NaN,100.0,-5201.0,-5201.0,-520.10,45693.0
1,2022-03-01 15:24:12,49350256,EURUSD,buy,10,1.11441,1.11341,1.11741,2022-03-01 17:10:21,1.11340,0.0,0.00,-1010.00,NaN,NaN,10000.0,-10.1,-5211.1,-1530.10,6369.0
2,2022-03-01 15:45:08,49351425,BITCOIN,sell,20,44477.04000,44 777.04,44 177.04,2022-03-01 16:33:16,44173.06000,0.0,0.00,6079.50,NaN,NaN,100.0,30398.0,25186.9,4549.40,2888.0
3,2022-03-03 08:07:11,49432596,EURGBP,buy,1,0.82804,0.82604,0.83104,2022-03-04 09:25:12,0.82604,0.0,- 5.35,-266.51,NaN,NaN,10000.0,-20.0,25166.9,4282.89,91081.0
4,2022-03-03 08:23:08,49432860,WTICOUSD,sell,5,112.39500,113.614,109.956,2022-03-03 09:26:06,113.61800,0.0,0.00,-6115.00,NaN,NaN,100.0,-122.3,25044.6,-1832.11,3778.0
5,2022-03-03 08:37:33,49433058,XPDUSD,sell,1,2681.38000,2 705.60,2 657.16,2022-03-03 09:04:31,2707.34000,0.0,0.00,-2596.00,NaN,NaN,100.0,-2596.0,22448.6,-4428.11,1618.0
6,2022-03-03 08:51:02,49433247,GBPUSD,buy,1,1.34097,1.33910,1.34237,2022-03-03 09:47:13,1.33908,0.0,0.00,-189.00,NaN,NaN,10000.0,-18.9,22429.7,-4617.11,3371.0
7,2022-03-03 08:57:11,49433357,USDMXN,buy,2,20.64708,20.58859,20.67110,2022-03-03 10:24:18,20.67181,0.0,0.00,239.26,NaN,NaN,10000.0,247.3,22677.0,-4377.85,5227.0
8,2022-03-03 08:59:04,49433389,BITCOIN,buy,10,43500.96000,43 025.18,43 867.18,2022-03-03 15:56:30,43869.41000,0.0,0.00,3684.50,NaN,NaN,100.0,36845.0,59522.0,-693.35,25046.0
9,2022-03-03 09:00:40,49433433,XPTUSD,sell,1,1082.82000,1 093.68,1 071.96,2022-03-03 10:57:49,1093.64000,0.0,0.00,-1082.00,NaN,NaN,100.0,-1082.0,58440.0,-1775.35,7029.0


In [21]:
f_estadisticas_ba(posiciones)

{'df_1_tabla':                       Valor                            Descripcion
 Medida                                                            
 Ops totales       15.000000                    Operaciones totales
 Ganadoras          6.000000                  Operaciones ganadoras
 Ganadoras_c        2.000000        Operaciones ganadoras de compra
 Ganadoras_v        4.000000         Operaciones ganadoras de venta
 Perdedoras         9.000000                 Operaciones perdedoras
 Perdedoras_c       4.000000       Operaciones perdedoras de compra
 Perdedoras_v       5.000000        Operaciones perdedoras de venta
 Mediana(Profit) -266.510000       Mediana de profit de operaciones
 Mediana(Pips)    -18.900000         Mediana de pips de operaciones
 r_efectividad      0.400000  Ganadoras Totales/Operaciones Totales
 r_proporcion       0.666667   Ganadoras Totales/Perdedoras Totales
 r_efectividad_c    0.133333  Ganadoras compras/Operaciones Totales
 r_efectividad_v    0.266667   Gan

## Tercer modulo

In [53]:
def f_be_de(param_data):
    # Dataframe con eventos ancla 
    operaciones_ancla = param_data.loc[param_data["Profit"] > 0]
    # Dataframe con operaciones restantes 
    operaciones_otras = param_data.loc[param_data["Profit"] <= 0]
    # Crear columna Ratio
    param_data['ratio'] = param_data['profit'] / param_data['profit_acum']) * 100
    
    for i in operaciones_ancla:
        if operaciones_anclas.loc[i, 'Closetime'] >= param_data.loc[j, 'Opentime'] and operaciones_anclas.loc[i, 'Closetime'] =< param_data.loc[j, 'Closetime']:
            
            